In [1]:
from py2neo import Graph, Node, Relationship, NodeMatcher
import py2neo

In [5]:
g = Graph('http://localhost:7474', user='neo4j', password='12345678', name='neo4j')   # 后面报错Cannot decode response content as JSON要加上name='neo4j

In [3]:
# g.run('match (n) detach delete n')

(No data)

# 创建一段关系链

In [6]:
test_node_1 = Node('Person', name='吴彦祖')
test_node_2 = Node('Person', name='周杰伦')
friend = Relationship(test_node_1, 'friend', test_node_2)
g.create(friend)

## 建立一个节点

In [4]:
test_node_1 = Node('Person', name='吴彦祖')
test_node_2 = Node('Person', name='周杰伦')


ProtocolError: Cannot decode response content as JSON

In [5]:
test_node_1['age']=18
test_node_1['sex']='男'

test_node_2['age']=16
test_node_2['sex']='女'

In [6]:
## 创建节点
g.create(test_node_1)
g.create(test_node_2)

In [7]:
## 覆盖节点
g.merge(test_node_1, "Person","name")
g.merge(test_node_2, "Person","name")

## 创建关系

In [8]:
friend = Relationship(test_node_1, 'friend', test_node_2)

In [9]:
g.merge(friend, "Person", "name")

## 查询节点

In [11]:
# 匹配所有节点
nodes = g.nodes.match()

for node in nodes:
    print(node)
    print(node.items())
    print(node.labels)

(_0:Person {name: '\u5434\u5f66\u7956'})
dict_items([('name', '吴彦祖')])
:Person
(_1:Person {name: '\u5468\u6770\u4f26'})
dict_items([('name', '周杰伦')])
:Person


In [18]:
# 精确查找
matcher = NodeMatcher(g)

# 返回第一个匹配的节点
node1 = matcher.match("Person", name="周杰伦").first() 
print(node1)
print(node1['name'])

print('\n')
# 返回多个匹配的节点
node1 = matcher.match("Person", name="周杰伦").all() 
print(node1)
print(node1[0])
print(node1[0]['name'])

(_1:Person {name: '\u5468\u6770\u4f26'})
周杰伦


[Node('Person', name='周杰伦')]
(_1:Person {name: '\u5468\u6770\u4f26'})
周杰伦


# 更新节点

先找出Nodes，在使用事务的push更新

In [19]:
from py2neo import Graph, Node, Relationship, NodeMatcher, Subgraph
g = Graph('http://localhost:7474', user='neo4j', password='12345678', name='neo4j')

tx = g.begin()
# 找到你要找的node
matcher = NodeMatcher(g)

# 修改单个节点
init_node = matcher.match("Person",name="周杰伦")
new_node = init_node.first()
new_node['name'] = '周捷纶'
sub = Subgraph(nodes=[new_node])
tx.push(sub)
tx.commit()

# 修改多个节点
# init_node = matcher.match("Person",name="周杰伦")
# new_node = []
# for node in init_node.all():
#     node['name'] = 'haha' + node['name']
#     new_node.append(node)
    
# sub = Subgraph(nodes=new_node)
# tx.push(sub)
# tx.commit()


C:\Users\19396\AppData\Local\Temp\ipykernel_24524\2661891343.py:14: DeprecationWarning: The transaction.commit() method is deprecated, use graph.commit(transaction) instead
  tx.commit()


# 删除节点/关系

In [ ]:
from py2neo import Graph, Node, Relationship, NodeMatcher, RelationshipMatcher
g = Graph('http://localhost:7474', user='neo4j', password='12345678', name='neo4j')

# 找到你要找的node
matcher = NodeMatcher(g)
r_matcher = RelationshipMatcher(g)

zjl = matcher.match("Person", name="周捷纶").first()
wyz = matcher.match("Person", name="吴彦祖").first()

relation = r_matcher.match(nodes=[zjl, wyz].first)

g.delete(relation) # 删除关系，节点也会无